# ДПО ВШЭ

## Современный анализ данных, глубокое обучение и приложения

## Задание на тему: "Рекомендательные системы"

В этом задании будем практиковаться в реализации рекомендательных систем.

Воспользуемся небольшим датасетом с Kaggle: [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [1]:
import numpy as np
import scipy
import pandas as pd
import math

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Часть 0. Загрузка данных

Загрузим [Deskdrop dataset](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop), включающийся в себе логи за 1 год платформы, где пользователи читают статьи.

Данные включают в себя 2 файла:  
- **shared_articles.csv**
- **users_interactions.csv**

Как можно догадаться, в одном описания самих статей (нам понадобятся в контентных моделях), а в другом логи пользователей.

#### shared_articles.csv

Так как в файле перечислены даже удалённые статьи, то мы их сразу удалим (на самом деле они могли бы быть нам полезны для подсчёта различных величин, хоть мы и не можем их рекомендовать).

In [2]:
articles_df = pd.read_csv('shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


#### users_interactions.csv

В колонке eventType описаны действия, которые могли совершать пользователи над статьёй:  
- VIEW
- LIKE
- COMMENT CREATED
- FOLLOW
- BOOKMARK

In [3]:
interactions_df = pd.read_csv('users_interactions.csv')
interactions_df.head(10)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
5,1465413742,VIEW,310515487419366995,-8763398617720485024,1395789369402380392,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR
6,1465415950,VIEW,-8864073373672512525,3609194402293569455,1143207167886864524,NaN,NaN,NaN
7,1465415066,VIEW,-1492913151930215984,4254153380739593270,8743229464706506141,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
8,1465413762,VIEW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
9,1465413771,VIEW,3064370296170038610,3609194402293569455,1143207167886864524,NaN,NaN,NaN


In [4]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

### Предобработка данных

В логах встречаются различные действия пользователей. Однако мы хотим работать лишь с одной величиной, характеризующей всё взаимодействие пользователя со статьёй. Предлагается задать действиям следующие веса:

In [17]:
def get_rating(value):
    event_type_strength = {
       'VIEW': 1.0,
       'LIKE': 2.0, 
       'BOOKMARK': 2.5, 
       'FOLLOW': 3.0,
       'COMMENT CREATED': 4.0,  
    }
    return event_type_strength[value]

Посчитайте числовую величину "оценки" пользователем статьи с указанными выше весами.

In [21]:
interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda i: get_rating(i))

In [23]:
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0


Ремендательные системы подвержены проблеме холодного старта. В рамках данного задания предлагается работать только с теми пользователями, которые взаимодействовали хотя бы с 5 материалом.

Оставьте только таких пользователей. Их должно остаться 1140.

In [24]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


In [26]:
users_with_enough_interactions_df.head()

,personId
0,-1007001694607905623
1,-1032019229384696495
2,-108842214936804958
3,-1119397949556155765
4,-1130272294246983140


Оставьте только те взаимодействия, которые касаются только отфильтрованных пользователей.

In [28]:
interactions_from_selected_users_df = interactions_df.loc[interactions_df['personId'].isin(users_with_enough_interactions_df['personId'])]

In [29]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

# interactions before: (72312, 9)
# interactions after: (69868, 9)


В данной постановке каждый пользователей мог взаимодействовать с каждой статьёй более 1 раза (как минимум совершая различные действия). Предлагается "схлопнуть" все действия в одно взаимодействие с весом, равным сумме весов. 

Однако полученное число будет в том числе тем больше, чем больше действий произвёл человек. Чтобы уменьшить разброс предлагается взять логарифм от полученного числа (можно придумыват другие веса действиям и по-другому обрабатывать значения).

Также сохраним последнее значение времени взаимодействия для разделениея выборки на обучение и контроль.

In [30]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(10)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324
5,-1007001694607905623,8729086959762650511,1.000000,1487240086
6,-1032019229384696495,-1006791494035379303,1.000000,1469129122
7,-1032019229384696495,-1039912738963181810,1.000000,1459376415
8,-1032019229384696495,-1081723567492738167,2.000000,1464054093
9,-1032019229384696495,-1111518890369033396,1.000000,1470109122


Разобьём выборку на обучение и контроль по времени.

In [31]:
split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 29329
# interactions on Test set: 9777


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказанями в виде списков.

In [35]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(10)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]
-123314413156450014,"[-1596335233529956315, -6133746337603876146, -...","[-14569272361926584, -3058031327323357308, -59..."
-1251984896177895077,"[-1596335233529956315, -2358756719610361882, -...","[-1452340812018195881, -14569272361926584, -41..."
-1257176162426022931,"[-1022885988494278200, -2479936301516183562, -...","[-1129449063360470561, -1181242543195516037, -..."
-1272364595235252747,"[-1297580205670251233, -133139342397538859, -6...","[-3302834510061927448, -6728844082024523434]"


## Часть 1: Baseline (модель по популярности)

Самой простой моделью рекомендаций (при этом достаточно сильной!) является модель, которая рекомендует наиболее популярные предметы. 

Предлагается реализовать её. Давайте считать, что рекомендуем мы по 10 материалов (такое ограничение на размер блока на сайте).

Посчитайте популярность каждой статьи, как сумму всех "оценок" взаимодействий с ней. Отсортируйте материалы по их популярности.

In [51]:
popular_content = (
    interactions_full_df
    .groupby('contentId')
    .eventStrength.agg('sum')
    .reset_index()
    .rename(columns={'eventStrength': 'totlaEventStrength'})
    .sort_values('totlaEventStrength', des)
    )

In [52]:
popular_content

,contentId,totlaEventStrength
2482,6541551984368289722,1.000000
485,-3854488592949785912,1.000000
2789,8233465999590288609,1.000000
2263,5271205718596251417,1.000000
278,-2587745393898245957,1.000000
1297,-8388400806941996172,1.000000
2790,8240934591713322028,1.000000
1302,-8418620743404378592,1.000000
151,-1804728901674946596,1.000000
1667,2051327203770554158,1.000000


Теперь необходимо сделать предсказания для каждого пользователя. Не забывайте, что надо рекомендовать то, что пользователь ещё не читал (для этого нужно проверить, что материал не встречался в true_train).

In [ ]:
top_k = 10

interactions['prediction_popular'] = # your code

Настало время оценить качество. Посчитайте precision@10 для каждого пользователя (доля угаданных рекомендаций). Усредните по всем пользователям. Везде далее будем считать эту же метрику.

In [ ]:
# your code

## Часть 2. Коллаборативная фильтрация.

Перейдём к более сложному механизму рекомендаций, а именно коллаборативной фильтрации. Суть коллаборативной фильтрации в том, что учитывается схожесть пользователей и товаров между собой, а не факторы, которые их описывают. 

__Предлагается на выбор реализовать один из двух подходов__: memory-based или модель со скрытыми переменные.

Для начала для удобства составим матрицу "оценок" пользователей. Нули будут обозначать отсутствие взаимодействия.

In [ ]:
ratings = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)

### Memory-based

Посчитайте схожести пользователей с помощью корреляции Пирсона. Для каждой пары учитываем только ненулевые значения.

Для скорости работы лучше переходить от pandas к numpy.

In [ ]:
ratings_m = ratings.as_matrix()

In [ ]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

for i in tqdm_notebook(range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
        
        # nonzero elements of two users
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        
        # continue if no intersection
        if np.sum(mask_uv) == 0:
            continue
            
        # get nonzero elements
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]
        
        # normalization
        # ...
        
        # for nonzero std
        if len(np.unique(ratings_v)) < 2 or len(np.unique(ratings_u)) < 2:
            continue
        
        similarity_users[i,j] = # your code
        similarity_users[j,i] = # your code

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для рекомендаций.

Для каждого пользователя:

1. Найдём пользователей с похожестью больше $\alpha$ на нашего пользователя.
2. Посчитаем для каждой статьи долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
3. Порекомендуем статьи с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

В нашем примере данных не очень много, поэтому возьмём $\alpha = 0$.

После того, как будут сделаны предсказания (новый столбец в interactions), посчитайте качество по той же метрике.

In [ ]:
# your code

interactions['prediction_user_based'] = # your code

### Модель со скрытыми переменными

Реализуем подход с разложением матрицы оценок. Для этого сделайте сингулярное разложение (svd в scipy.linalg), на выходе вы получите три матрицы.

In [ ]:
# your code

Значения у матрицы с сингулярными числами отсортированы по убыванию. Допустим мы хотим оставить только первые 100 компонент (и получить скрытые представления размерности 100). Для этого необходимо оставить 100 столбцов в матрице U, оставить из sigma только первые 100 значений (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Перемножьте преобразованные матрицы ($\hat{U}, \hat{sigma}, \hat{V^T}$), чтобы получить восстановленную матрицу оценок.

In [ ]:
# your code

Посчитайте качество аппроксимации матрицы по норме Фробениуса (среднеквадратичную ошибку между всеми элементами соответствующими элементами двух матриц). Сравните его с простым бейзлайном с константным значением, равным среднему значению исходной матрицы. У аппроксимации ошибка должна получиться ниже.

In [ ]:
# your code

Теперь можно делать предсказания по матрице. Сделайте их (не забывайте про то, что уже было просмотрено пользователем), оцените качество. Для этого необходимо для каждого пользователя найти предметы с наибольшими оценками в восстановленной матрице.

In [ ]:
# your code

До этих пор мы не проводили никаких преобразований с матрицей оценок. Отцентрируйте все ненулевые (!) значения по каждому пользователю. Сделайте предсказания, посчитайте качество.

In [ ]:
# your code

### Факторизационные машины (опционально)

Попробуем факторизационные машины из библиотеки pyFM (так как можно работать прямо из питона). https://github.com/coreylynch/pyFM

In [ ]:
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer

Перейдём к обобщению матричных разложений — факторизационным машинам, которые работают могут работать с контентной информацией. Вспомним, какие данные у нас изначально были:

В факторизационную машину можно загрузить "айдишники" пользователей и статей (то есть сделать аналог коллаборативной фильтрации) и одновременно различные признаки.

Удобно обрабатывать категориальные переменные (id и другие) можно с помощью DictVectorizer. Например, процесс может выглядить вот так:

In [ ]:
# train = [
#     {"user": "1", "item": "5", "age": 19},
#     {"user": "2", "item": "43", "age": 33},
#     {"user": "3", "item": "20", "age": 55},
#     {"user": "4", "item": "10", "age": 20},
# ]
# v = DictVectorizer()
# X = v.fit_transform(train)
# y = np.repeat(1.0, X.shape[0])
# fm = pylibfm.FM()
# fm.fit(X,y)
# fm.predict(v.transform({"user": "1", "item": "10", "age": 24}))

Сгенерируйте таблицу с признаками в таком виде, где будут id пользователя, статьи и автора статьи и несколько признаков, которые вы сможете придумать. В качестве целевой переменной возьмите "силу" взаимодействия пользователя с каждой статьёй (помним, что у нас там все примеры по сути положительные). Запустите обучение модели на несколько итераций и сделайте предсказания. Какое качество удаётся достич? 

In [ ]:
train_data = []
test_data = []

for i in tqdm_notebook(range(len(interactions_train_df))):
    features = {}
    features['personId'] = str(interactions_test_df.iloc[i].personId)
    features['contentId'] = str(interactions_test_df.iloc[i].contentId)
    
    # ... 
    
    train_data.append(features)
    
# and test set too

Векторизуем, получим разреженные матрицы.

In [ ]:
dv = DictVectorizer()

train_features = dv.fit_transform(train_data)
test_features = dv.transform(test_data)

In [ ]:
train_features

In [ ]:
fm = pylibfm.FM(num_factors=10, num_iter=30, task='regression')
fm.fit(train_features, y_train)

In [ ]:
# your code

Попробуйте добавить случайные негативные примеры из статей, с которыми пользователь не взаимодействовал. Какое качество удалось достичь?

In [ ]:
# your code

## Часть 3. Контентные  модели

В этой части реализуем альтернативных подход к рекомендательным системам — контентные модели.

Теперь мы будем оперировать не матрицей с оценками, а классической для машинного обучения матрицей объекты-признаки. Каждый объект будет характеризовать пару user-item и содержать признаки, описывающие как пользователя, так и товар. Кроме этого признаки могут описывать и саму пару целиком.

Матрица со всеми взаимодействиями уже получена нами на этапа разбиения выборки на 2 части. 

Придумаем и добавим признаков о пользователях и статьях. Сначала добавим информацию о статьях в данные о взаимодействиях.

In [ ]:
interactions_train_df = interactions_train_df.merge(articles_df, how='left', on='contentId')
interactions_test_df = interactions_test_df.merge(articles_df, how='left', on='contentId')

In [ ]:
# first feature index
features_start = len(interactions_train_df.columns)

После обучения модели нам придётся делать предсказания на тестовой выборке для всех возможных пар статья-пользователь. Подготовим такую матрицу, чтобы параллельно посчитать признаки для неё.

In [ ]:
test_personId = np.repeat(interactions.index, len(articles_df)) 
test_contentId = list(articles_df.contentId) * len(interactions)

test = pd.DataFrame(
    np.array([test_personId, test_contentId]).T,
    columns=['personId', 'contentId'])
test = test.merge(articles_df, how='left', on='contentId')

test.head()

Добавьте признаки-индикаторы возможных значений contentType.

In [ ]:
articles_df.contentType.unique()

In [ ]:
interactions_train_df['is_HTML'] = interactions_train_df.contentType == 'HTML'
# your code

Добавьте признаки "длина названия" и "длина текста".

In [ ]:
# your code

Добавьте признаки-индикаторы языка.

In [ ]:
# your code

Обучим на полученных признаках градиентный бустинг.

In [ ]:
import lightgbm 

regressor = lightgbm.LGBMRegressor()
regressor.fit(interactions_train_df[interactions_train_df.columns[features_start:]],
              interactions_train_df.eventStrength)

Сделайте предсказания на тестовой выборке, сформируйте из них рекомендации. Оцените их качество.

In [ ]:
# your code

Категориальные переменные с большим количеством значений можно закодировать с помощью mean-target кодирования. Закодируйте так id статьи и пользователя. Обучите новую модель и оцените качество.

In [ ]:
# your code

Добавим негативных примеров в выборку — для каждого пользователя выберем случайные статьи, которые он не читал и отметим их нулём. Попробуйте добавить негативных примеров в обучающую выборку.

In [ ]:
# your code